In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from imblearn.under_sampling import NearMiss
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from numpy import set_printoptions
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn import svm
from sklearn import tree
from sklearn.feature_selection import SelectPercentile, f_classif
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn import metrics
import lime.lime_tabular as lm#the type of LIIME analysis we’ll do
import shap #SHAP package
from sklearn.inspection import permutation_importance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19final-dataset/both_test_unbalanced.csv
/kaggle/input/covid19final-dataset/pcr_unbalanced.csv
/kaggle/input/covid19final-dataset/rapid_balanced.csv
/kaggle/input/covid19final-dataset/both_test_balanced.csv
/kaggle/input/covid19final-dataset/rapid_unbalanced.csv
/kaggle/input/covid19final-dataset/pcr_balanced.csv


In [ ]:
data_rapid = pd.read_csv("/kaggle/input/covid19final-dataset/rapid_balanced.csv")
data_rapid=data_rapid.astype(int)

data_pcr= pd.read_csv("/kaggle/input/covid19final-dataset/pcr_balanced.csv")
data_pcr=data_pcr.astype(int)

data_both= pd.read_csv("/kaggle/input/covid19final-dataset/both_test_balanced.csv")
data_both=data_both.astype(int)


x_col=data_rapid.iloc[:,:10].columns

In [ ]:
def RandomForest_classif(x_train,y_train,param):
    #Classification
    
    
    clf= RandomForestClassifier(**param)
    clf=clf.fit(x_train, y_train)
    
    return clf

def Kneighbors_classif(x_train,y_train,param):
    #Classification
    

    clf= KNeighborsClassifier(**param)
    clf= clf.fit(x_train, y_train)
    
    return clf

def DecisionTree_classif(x_train,y_train,param):
    #Classification
    
    clf = tree.DecisionTreeClassifier(**param)
    clf = clf.fit(x_train,y_train)
    
    return clf

def mpl_classif(x_train,y_train,param):
    
    clf =  MLPClassifier(**param)
    clf=clf.fit(x_train,y_train)
    
    return clf

def gb_classif(x_train,y_train,param):
    
    clf=GradientBoostingClassifier(**param)
    clf=clf.fit(x_train, y_train)
    
    return clf

def xgb_classif(x_train,y_train,param):
    clf = xgb.XGBClassifier(**param)
    
    return clf.fit(x_train, y_train)

def svc_classif(x_train,y_train,param):
    
    regr = svm.SVC(**param)
    regr=regr.fit(x_train, y_train)

    return regr

def lg_with_regu_classif(x_train,y_train,param):
    
    clf = LogisticRegression(** param)
    clf=clf.fit(x_train,y_train)
    
    return clf
def lg_without_regu_classif(x_train,y_train,param):
    
    clf = LogisticRegression(** param)
    clf=clf.fit(x_train,y_train)
    
    return clf




In [ ]:
def calculate_feature_importance(model) :
    feat_importances = pd.Series(model.feature_importances_, index=x_col)
    # determine top most important features
    return feat_importances.nlargest(len(x_col))

def calculate_feature_importance2(model) :
    feat_importances = pd.Series(model.best_estimator_.feature_importances_, index=x_col)
    # determine top most important features
    return feat_importances.nlargest(len(x_col))


In [ ]:
def feature_importance(X_train, X_val, y_train, y_val,model,label):
    
    r = permutation_importance(model, X_val, y_val,n_repeats=50,random_state=0)
    for i in r.importances_mean.argsort()[::-1]:
       
            print(f"{X_train.columns[i]:<8}" f"{r.importances_mean[i]:.3f}"
                  f" +/- {r.importances_std[i]:.3f}")
    sorted_idx = r.importances_mean.argsort()

    fig, ax = plt.subplots()
    ax.boxplot(r.importances[sorted_idx].T,
               vert=False, labels=X_train.columns[sorted_idx])
    ax.set_title("Permutation Importances:"+label)
    fig.tight_layout()
    plt.show()
    


In [ ]:
def plot_feature_importance(importance,color_importace,classif_name): 
    indices =importance.values
    features = importance.index
    plt.title(classif_name+'\nTop Feature Importances')
    plt.barh(range(len(indices)), indices, color=color_importace, align='center')
    plt.yticks(range(len(indices)), features)
    plt.xlabel('Relative Importance')
    plt.show()

# rapid_test_balanced

In [ ]:
x_rapid_balan=data_rapid.iloc[:,:10]
y_rapid_balan=data_rapid['Class']


In [ ]:
lista_param_rapid=[
             {},
             {},
             {'kernel': 'rbf', 'C': 6,'probability':True},
             {'weights': 'distance', 'n_neighbors': 15,'n_jobs': None, 'leaf_size': 5, 'algorithm': 'ball_tree'},
             {'n_estimators':500,'max_depth':5},
             {'solver': 'liblinear', 'penalty': 'l2','C': 0.1},
             {'solver': 'liblinear', 'penalty': 'l2','C': 1.0},
             {'solver': 'adam', 'random_state': 1, 'max_iter': 1200, 'learning_rate': 'adaptive','alpha': 0.0001, 'activation': 'relu'},
             {'n_estimators':300,'max_depth':9,'min_child_weight': 2}]

In [ ]:
X_train, X_val, y_train, y_val  = train_test_split(x_rapid_balan,y_rapid_balan, 
                                  test_size=0.5,random_state=0,stratify=y_rapid_balan)
model=Kneighbors_classif(X_train,y_train,lista_param_rapid[3])
feature_importance(X_train, X_val, y_train, y_val,model,'KNN')


In [ ]:
model= svc_classif(X_train,y_train,lista_param_rapid[2])
feature_importance(X_train, X_val, y_train, y_val,model,'SVM')



In [ ]:
model=RandomForest_classif(X_train,y_train,lista_param_rapid[1])
feature_importance(X_train, X_val, y_train, y_val,model,'RF')

In [ ]:
model_rf= RandomForest_classif(x_rapid_balan,y_rapid_balan,lista_param_rapid[1])
importance_rf=calculate_feature_importance(model_rf)
plot_feature_importance(importance_rf,'pink','Random Forest')


In [ ]:
importance_rf

In [ ]:
model=DecisionTree_classif(X_train,y_train,lista_param_rapid[0])
feature_importance(X_train, X_val, y_train, y_val,model,'DT')

In [ ]:
model_dt= DecisionTree_classif(x_rapid_balan,y_rapid_balan,lista_param_rapid[0])
importance_dt=calculate_feature_importance(model_dt)
plot_feature_importance(importance_dt,'purple','Decision Tree')

In [ ]:
importance_dt

In [ ]:
model=gb_classif(X_train,y_train,lista_param_rapid[4])
feature_importance(X_train, X_val, y_train, y_val,model,'GBM')

In [ ]:
model_gb=gb_classif(x_rapid_balan,y_rapid_balan,lista_param_rapid[4])
importance_gb=calculate_feature_importance(model_gb)
plot_feature_importance(importance_gb,'green','Gradient Boosting')


In [ ]:
importance_gb

In [ ]:
model=xgb_classif(X_train,y_train,lista_param_rapid[8])
feature_importance(X_train, X_val, y_train, y_val,model,'Xgboos')

In [ ]:
model_xgb=xgb_classif(x_rapid_balan,y_rapid_balan,lista_param_rapid[8])
importance_xgb=calculate_feature_importance(model_xgb)
plot_feature_importance(importance_xgb,'red','Xgboos')


In [ ]:
importance_xgb

In [ ]:
model=mpl_classif(X_train,y_train,lista_param_rapid[7])
feature_importance(X_train, X_val, y_train, y_val,model,'MLP')


In [ ]:
model=lg_with_regu_classif(X_train,y_train,lista_param_rapid[5])
feature_importance(X_train, X_val, y_train, y_val,model,'LRR')



In [ ]:
model=lg_without_regu_classif(X_train,y_train,lista_param_rapid[6])
feature_importance(X_train, X_val, y_train, y_val,model,'LR')

# pcr_test_balanced

In [ ]:
lista_param_pcr=[
{},
{},
{'kernel': 'rbf', 'C': 7,'probability':True},
{'weights': 'distance', 'n_neighbors': 5, 'n_jobs': None, 'leaf_size': 3, 'algorithm': 'kd_tree'},
{'n_estimators':500,'max_depth':5},
{'solver': 'liblinear', 'penalty': 'l2','C': 0.1},
{'solver': 'liblinear', 'penalty': 'l2','C': 1.0},
{'max_iter': 1800,'solver':'lbfgs', 'alpha':1e-5, 'random_state':42},
{'n_estimators':300,'max_depth':9,'min_child_weight': 2}]

In [ ]:
x_pcr=data_pcr.iloc[:,:10]
y_pcr=data_pcr['Class']
X_train, X_val, y_train, y_val  = train_test_split(x_pcr,y_pcr,test_size=0.5,random_state=0,stratify=y_pcr)

In [ ]:
model=Kneighbors_classif(X_train,y_train,lista_param_rapid[3])
feature_importance(X_train, X_val, y_train, y_val,model,'KNN')


In [ ]:
model= svc_classif(X_train,y_train,lista_param_rapid[2])
feature_importance(X_train, X_val, y_train, y_val,model,'SVM')

In [ ]:
model=RandomForest_classif(X_train,y_train,lista_param_rapid[1])
feature_importance(X_train, X_val, y_train, y_val,model,'RF')

In [ ]:
model_rf= RandomForest_classif(x_pcr,y_pcr,lista_param_pcr[1])
importance_rf=calculate_feature_importance(model_rf)
plot_feature_importance(importance_rf,'pink','Random Forest')


In [ ]:
importance_rf

In [ ]:
model=DecisionTree_classif(X_train,y_train,lista_param_rapid[0])
feature_importance(X_train, X_val, y_train, y_val,model,'DT')

In [ ]:
model_dt= DecisionTree_classif(x_pcr,y_pcr,lista_param_pcr[0])
importance_dt=calculate_feature_importance(model_dt)
plot_feature_importance(importance_dt,'purple','Decision Tree')

In [ ]:
importance_dt

In [ ]:
model=gb_classif(X_train,y_train,lista_param_rapid[4])
feature_importance(X_train, X_val, y_train, y_val,model,'GBM')

In [ ]:
model_gb=gb_classif(x_pcr,y_pcr,lista_param_pcr[4])
importance_gb=calculate_feature_importance(model_gb)
plot_feature_importance(importance_gb,'green','Gradient Boosting')


In [ ]:
importance_gb

In [ ]:
model=xgb_classif(X_train,y_train,lista_param_rapid[8])
feature_importance(X_train, X_val, y_train, y_val,model,'Xgboos')

In [ ]:
model_xgb=xgb_classif(x_pcr,y_pcr,lista_param_pcr[8])
importance_xgb=calculate_feature_importance(model_xgb)
plot_feature_importance(importance_xgb,'red','Xgboos')

In [ ]:
importance_xgb

In [ ]:
model=mpl_classif(X_train,y_train,lista_param_rapid[7])
feature_importance(X_train, X_val, y_train, y_val,model,'MLP')

In [ ]:
model=lg_with_regu_classif(X_train,y_train,lista_param_rapid[5])
feature_importance(X_train, X_val, y_train, y_val,model,'LRR')

In [ ]:
model=lg_without_regu_classif(X_train,y_train,lista_param_rapid[6])
feature_importance(X_train, X_val, y_train, y_val,model,'LRR')

# both_test_balanced

In [ ]:
lista_param_both=[{},{},{'kernel': 'rbf', 'C': 9},
             {'weights': 'distance','n_neighbors': 8, 'n_jobs': -1,'leaf_size': 5, 'algorithm': 'brute'},
             {'n_estimators':500,'max_depth':5},
             {'solver': 'liblinear', 'penalty': 'l2','C': 0.1},
             {'solver': 'liblinear', 'penalty': 'l2','C': 1.0},
             {'max_iter': 1200,'solver':'lbfgs', 'alpha':1e-5, 'random_state':42},
            {'n_estimators':300,'max_depth':9,'min_child_weight': 2}]

In [ ]:
x_both=data_both.iloc[:,:10]
y_both=data_both['Class']
X_train, X_val, y_train, y_val  = train_test_split(x_both,y_both, 
                                  test_size=0.5,random_state=0,stratify=y_both)

In [ ]:
model=Kneighbors_classif(X_train,y_train,lista_param_rapid[3])
feature_importance(X_train, X_val, y_train, y_val,model,'KNN')


In [ ]:
model= svc_classif(X_train,y_train,lista_param_rapid[2])
feature_importance(X_train, X_val, y_train, y_val,model,'SVM')

In [ ]:
model=RandomForest_classif(X_train,y_train,lista_param_rapid[1])
feature_importance(X_train, X_val, y_train, y_val,model,'RF')

In [ ]:
model_rf= RandomForest_classif(x_both,y_both,lista_param_both[1])
importance_rf=calculate_feature_importance(model_rf)
plot_feature_importance(importance_rf,'pink','Random Forest')

In [ ]:
importance_rf

In [ ]:
model=DecisionTree_classif(X_train,y_train,lista_param_rapid[0])
feature_importance(X_train, X_val, y_train, y_val,model,'DT')

In [ ]:
model_dt= DecisionTree_classif(x_both,y_both,lista_param_both[0])
importance_dt=calculate_feature_importance(model_dt)
plot_feature_importance(importance_dt,'purple','Decision Tree')

In [ ]:
importance_dt

In [ ]:
model=gb_classif(X_train,y_train,lista_param_rapid[4])
feature_importance(X_train, X_val, y_train, y_val,model,'GBM')

In [ ]:
model_gb=gb_classif(x_both,y_both,lista_param_both[4])
importance_gb=calculate_feature_importance(model_gb)
plot_feature_importance(importance_gb,'green','Gradient Boosting')


In [ ]:
importance_gb

In [ ]:
model=xgb_classif(X_train,y_train,lista_param_rapid[8])
feature_importance(X_train, X_val, y_train, y_val,model,'Xgboos')

In [ ]:
model_xgb=xgb_classif(x_both,y_both,lista_param_both[8])
importance_xgb=calculate_feature_importance(model_xgb)
plot_feature_importance(importance_xgb,'red','Xgboos')

In [ ]:
importance_xgb

In [ ]:
model=mpl_classif(X_train,y_train,lista_param_rapid[7])
feature_importance(X_train, X_val, y_train, y_val,model,'MLP')

In [ ]:
model=lg_with_regu_classif(X_train,y_train,lista_param_rapid[5])
feature_importance(X_train, X_val, y_train, y_val,model,'LRR')

In [ ]:
model=lg_without_regu_classif(X_train,y_train,lista_param_rapid[6])
feature_importance(X_train, X_val, y_train, y_val,model,'LRR')